# MHPI hydroDL2.0 Tutorial: **dHBV1.1p**
---

This is a basic implementation of the generic differentiable modeling framework `dMG` using the HBV1.1p hydrology model
plugin from the `hydroDL2.0` repository.



Last Revision: 30 Oct. 2024

Authors: Leo Lonzarich

---

## 1. Basic Hands-off Deployment:

In this first demonstration, we show how `dMG` using a HBV1.0 or HBV1.1p physics model backbone from `hydroDL2` can be operated
in a  few steps. These are outlined as follows:

0. First, ensure that you have the correct *env* configured. To avoid manually downloading required Python packages,
create a `hydrodl` env using 

    `conda env create -f envs/hydrodl_env.yaml`.

    Once activated, confirm PyTorch installed correctly with `torch.cuda.is_available()`. If this reports false, try
    - `conda uninstall pytorch`
    - `conda install pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia`

1. Set your desired model and experiment configuration settings within a *yaml* config file.
    - For this tutorial, you can find this config located at `generic_diffModel/example/conf/dhbv_11p_config.yaml`. Note,
    this yaml is configured to reproduce dHBV1.1p benchmarks for 531 CAMELS basins, trained and tested for 9 and 10 years,
    respectively
    - For normal operation of `dMG`, however, see `generic_diffModel/conf/config.yaml`.
2. Either run `python dMG/__main__.py` in your terminal, or (recommended) run the contents of `__main__.py` in the cells below.
    - This will parse your config into a dictionary, load the HBV1.1p hydrology model, and begin training or testing.


### 1.1 Create Configurations Dictionary

- `dhbv_config.yaml` for HBV1.0
- `dhbv_11p_config.yaml` for HBV1.1p

The first cell below will convert the configurations yaml file into a key-indexed dictionary, with keys being the
config settings. 

That is, if `mode: train` is set in the config file, the dictionary will yield `config['mode'] == 'train'`.
Similarly, `training: start_time: 1999/10/01` is equivalent to `config['training']['start_time'] == '1999/10/01'`.


In [ ]:
## Load in the dMG configuration file with dHBV1.1p options:
import sys
import hydra
from omegaconf import DictConfig, OmegaConf
sys.path.append('../../dMG') # Add the root directory of dMG to the path


# Example configs stored in /example/conf
CONFIG_PATH = '../conf'
CONFIG_NAME = 'dhbv_11p_config'



def load_config(config_path: str, config_name: str) -> DictConfig:
    """ Initialize Hydra and parse model configuration yaml(s) into config dict. """
    with hydra.initialize(config_path=config_path, version_base='1.3'):
        config = hydra.compose(config_name=config_name)
   
    config_dict = OmegaConf.to_container(config, resolve=True)
    return config_dict

config = load_config(CONFIG_PATH, CONFIG_NAME)

### 1.2 Run `__main__.py` with Configurations

This code instantiates a model Trainer which will train or test a model per the user's specification in the config. Note
that `__main__.py` is trimmed-down here to illustrate it's primary objective. The trainer performs the following
functions:
- CAMELS data will be loaded and preprocessed,
- A differenial model object with the HBV1.1p backbone will be created, and 
- An optimizer and loss function will be initialized.

These and other details/structure of `dMG` will be illustrated in the second part of this tutorial.


In [ ]:
import torch
import logging
from typing import Any, Dict
from conf.config import ModeEnum
from trainers import build_handler
from core.utils import (create_output_dirs, set_randomseed, set_system_spec,
                        print_config)

log = logging.getLogger(__name__)



def run_train_test(config_dict: Dict[str, Any]) -> None:
    """
    Run training and testing as one experiment.
    """
    # Training
    config_dict['mode'] = ModeEnum.train
    train_experiment_handler = build_handler(config_dict)
    train_experiment_handler.run()

    # Testing
    config_dict['mode'] = ModeEnum.test
    test_experiment_handler = build_handler(config_dict)            
    test_experiment_handler.dplh_model_handler = train_experiment_handler.dplh_model_handler
    test_experiment_handler.run()


def run_experiment(config_dict: Dict[str, Any]) -> None:
    """ Run an experiment based on the mode specified in the configuration. """
    experiment_handler = build_handler(config_dict)
    experiment_handler.run()



# Set device, dtype, output directories, and random seed.
set_randomseed(config['random_seed'])

config['device'], config['dtype'] = set_system_spec(config['gpu_id'])
config = create_output_dirs(config)

log.info(f"RUNNING MODE: {config['mode']}")
print_config(config)

# Run training and testing together, or one at a time.
if config['mode'] == ModeEnum.train_test:
    run_train_test(config)

else:
    run_experiment(config)

torch.cuda.empty_cache()

### 1.3 Get Results of Tested Model

If you have run testing on a trained model and want to view the results, you can find a `mstd.csv` file in your model
directory, which will give you the statistics on your model's performance. 

*Graphical visualizations of model output will be supported in a future updated.*


---

## 2. Breakdown of Intermediate Steps: Training

In this example, we break down dHBV1.1p differentiable model training in `dMG` by exposing the internals of the Trainer.
(**Note**, we are bypassing `__main__.py` in this part since it simply runs the Trainer.)

### 2.1 Create Configurations Dictionary

Once again, begin by creating a configurations dictionary.

In [ ]:
## Load in the dMG configuration file with dHBV1.1p options:
import sys
import hydra
from omegaconf import DictConfig, OmegaConf
sys.path.append('../../dMG') # Add the root directory of dMG to the path

from core.utils import create_output_dirs, set_system_spec



# Example configs stored in /example/conf
CONFIG_PATH = '../conf'
CONFIG_NAME = 'dhbv_11p_config'



def load_config(config_path: str, config_name: str) -> DictConfig:
    """ Initialize Hydra and parse model configuration yaml(s) into config dict. """
    with hydra.initialize(config_path=config_path, version_base='1.3'):
        config = hydra.compose(config_name=config_name)
   
    config_dict = OmegaConf.to_container(config, resolve=True)

    config_dict['device'], config_dict['dtype'] = set_system_spec(config_dict['gpu_id'])
    config_dict = create_output_dirs(config_dict)

    return config_dict

config = load_config(CONFIG_PATH, CONFIG_NAME)

### 2.2 Load the Data

In this tutorial, we work with either 671 or 531 CAMELS basins.

MHPI Team
- For immediate access to the CAMELS train/test data files, run this tutorial on Suntzu server. Data paths are already
preconfigured for this server so nothing further needs to be done. See path specs here: 
`generic_diffModel/dMG/conf/observations/camels_531.yaml`.

Else
- Expanded instruction will be added for obtaining this data once `hydro_data` and `dMG` reach v1.0 release.



In [ ]:
from core.data.dataset_loading import get_data_dict

dataset_dict, config = get_data_dict(config, train=True)

### 2.3 Initialize model, optimizer and loss function

These are the auxillary tasks completed by the Trainer before beginning the training loop.


#### 2.3.1 Init Differentiable Model

In [ ]:
import torch.nn
from models.neural_networks.lstm_models import CudnnLstmModel
from models.neural_networks.mlp_models import MLPmul
from hydroDL2 import load_model

model_name = config['phy_model']['models'][0]



def init_phy_model(config, model_name):
    """Initialize the physics model."""
    if model_name == 'HBV':
        hydro_model = load_model('HBV')
        return hydro_model(config)
    elif model_name == 'HBV_11p':
        hydro_model = load_model('HBV_11p')
        return hydro_model()
    else:
        raise ValueError(model_name, "is not a valid physics model.")


def init_pnn_model(config, phy_model):
    """Initialize the pNN model."""
    n_forc = len(config['observations']['nn_forcings'])
    n_attr = len(config['observations']['nn_attributes'])
    n_model_params = len(phy_model.parameters_bound)
    n_rout_params = len(phy_model.conv_routing_hydro_model_bound)
    
    nx = n_forc + n_attr
    ny = config['dpl_model']['nmul'] * n_model_params

    if config['routing_hydro_model'] == True:
        ny += n_rout_params
    
    if config['pnn_model']['model'] == 'LSTM':
        nn_model = CudnnLstmModel(
            nx=nx,
            ny=ny,
            hiddenSize=config['pnn_model']['hidden_size'],
            dr=config['pnn_model']['dropout']
        )
    elif config['pnn_model']['model'] == 'MLP':
        nn_model = MLPmul(
            config,
            nx=nx,
            ny=ny
        )
    else:
        raise ValueError(config['pnn_model'], " not supported.")
    
    return nn_model
    


# 1. Initialize the physics model
phy_model = init_phy_model(config, model_name)

# 2. Initialize the parameterization neural network (pNN) model.
pnn_model = init_pnn_model(config, phy_model)

In [ ]:
# Method for extracting the physics model parameters from the pNN output. (Not necessary to expand)
def breakdown_params(config, phy_model, params_all):
    """Extract physics model parameters from pNN output."""
    n_model_params = len(phy_model.parameters_bound)
    n_rout_params = len(phy_model.conv_routing_hydro_model_bound)

    ny = config['dpl_model']['nmul'] * n_model_params
    if config['routing_hydro_model'] == True:
        ny += n_rout_params

    params_dict = dict()
    params_hydro_model = params_all[:, :, :ny]

    # Physics model params.
    params_dict['hydro_params_raw'] = torch.sigmoid(
        params_hydro_model[:, :, :len(phy_model.parameters_bound) * config['dpl_model']['nmul']]).view(
        params_hydro_model.shape[0], params_hydro_model.shape[1], len(phy_model.parameters_bound),
        config['dpl_model']['nmul'])
    
    # Routing params
    if config['routing_hydro_model'] == True:
        params_dict['conv_params_hydro'] = torch.sigmoid(
            params_hydro_model[-1, :, len(phy_model.parameters_bound) * config['dpl_model']['nmul']:])
    else:
        params_dict['conv_params_hydro'] = None
    return params_dict

In [ ]:
# 3. Define the differentiable model object and a basic forward method.
class dHBV(torch.nn.Module):
    def __init__(self, phy_model, pnn_model):
        super(dHBV, self).__init__()
        self.pnn_model = pnn_model
        self.phy_model = phy_model

    def forward(self, config, x_dict):
        """Basic Forward method.
        
        Parameters
        ----------
        x_dict : dict
            Dictionary containing the input data for the model.
        """
        # Forward pNN model;
        # Take array of predicted params from pNN and unpack into dictionary.
        params_all = self.pnn_model(x_dict['inputs_nn_scaled'])
        params_dict = breakdown_params(config, self.phy_model, params_all)
        
        # Forward physics model
        flow_out = self.phy_model(
            x_dict['x_hydro_model'],
            params_dict['hydro_params_raw'],
            config,
            static_idx=config['phy_model']['stat_param_idx'],
            warm_up=config['phy_model']['warm_up'],
            routing=config['routing_hydro_model'],
            conv_params_hydro=params_dict['conv_params_hydro']
        )

        # Baseflow index percentage;
        # Using two deep groundwater buckets: gwflow & bas_shallow
        if 'bas_shallow' in flow_out.keys():
            baseflow = flow_out['gwflow'] + flow_out['bas_shallow']
        else:
            baseflow = flow_out['gwflow']
        flow_out['BFI_sim'] = 100 * (torch.sum(baseflow, dim=0) / (
                torch.sum(flow_out['flow_sim'], dim=0) + 0.00001))[:, 0]

        return flow_out

In [ ]:
# 4. Initialize the differentiable model object, and assign to a GPU.
dpl_model = dHBV(phy_model, pnn_model).to(config['device'])
print(dpl_model)

#### 2.3.2 Init Optimizer

NOTE: currently a workaround, but will give a descriptive perscription in future updates.

The model handler object contains an optimizer and loss function that can be used for training. 

In [ ]:
from models.model_handler import ModelHandler

model_handler = ModelHandler(config)

# Pass the dpl_model into the model_handler.
model_handler.model_dict = {model_name: dpl_model}


# Get the learnable pNN parameters from the dpl_model for the optimizer.
model_params = dpl_model.parameters()
model_handler.all_model_params = model_params


# Initialize the optimizer (torch Adadelta).
model_handler.init_optimizer()
optim = model_handler.optim

#### 2.3.3 Init Loss Function

NOTE: currently a workaround with model handler, but will updated shortly.

In [ ]:
# Initialize the loss function (named in your configuration file).
model_handler.init_loss_func(dataset_dict['obs'])
loss_func = model_handler.loss_func

# See that the corerct loss function is loaded.
print(f"Config setting: {config['loss_function']['model']}")
print(f"Loaded loss function: {model_handler.loss_func}")

### 2.4 Train the Model


In [ ]:
import tqdm
from core.data import n_iter_nt_ngrid, take_sample_train
from core.utils import save_model



# Setup training grid.
ngrid_train, minibatch_iter, nt = n_iter_nt_ngrid(
    dataset_dict['inputs_nn_scaled'], config['train_t_range'], config
    )

# Start of training.
for epoch in range(1, config['train']['epochs'] + 1):
    ep_loss_dict = {model_name: 0.0}  # Initialize loss to zero.

    # Work through training data in batches.
    prog_str = f"Epoch {epoch}/{config['train']['epochs']}"
    
    for i in tqdm.tqdm(range(1, minibatch_iter + 1), desc=prog_str,
                        leave=False, dynamic_ncols=True):
        
        # Take a sample of the training data for the batch.
        dataset_dict_sample = take_sample_train(config, dataset_dict,
                                                ngrid_train, nt)

        # Forward pass through dPL model.
        model_preds = dpl_model.forward(config, dataset_dict_sample)        
        
        # Calculate loss.
        model_handler.flow_out_dict = {model_name: model_preds}
        model_handler.dataset_dict_sample = dataset_dict_sample

        hydro_loss, ep_loss_dict = model_handler.calc_loss(ep_loss_dict)
        
        total_loss = hydro_loss
        total_loss.backward()
        optim.step()
        optim.zero_grad()

    # Save the model every save_epoch (set in the confic).
    if epoch % config['train']['save_epoch'] == 0:
        save_model(config, dpl_model, model_name, epoch)